### Tili-toli játék

1. lépés: készíts programot, ami megoldja a [tizenötös játékot](https://hu.wikipedia.org/wiki/Tizen%C3%B6t%C3%B6s_j%C3%A1t%C3%A9k). Bemenetként tetszőleges kezdeti konfigurációt át lehessen adni. Egy lehetséges stratégia: 
 - rekurzívan járjuk be a tologatással elérhető játékhelyzeteket (minden helyzetben 2, 3 vagy 4 féle továbblépés lehetséges)
 - minden állásból előállítjuk az elérhető állásokat: a bejárás sorrendjét ekkor az határozza meg, hogy melyik állásban lesz kevesebb azon elemek száma, melyek nincsenek a helyükön (ha ez 0, akkor végeztünk)

 - tartsuk nyilván egy adatszerkezetben (pl `set`) a már bejárt állásokat, hogy elkerüljük a felesleges ismétlődést

2. lépés: tölts be egy általad választott képet a játékhoz: vágd fel a képet $4\times4$ azonos méretű darabra, azonosítsd sorfolytonosan az első 15 darabot az $1,2,\ldots,15$ számokkal. Rajzold ki a megoldáshoz vezető lépések sorozatát a képdarabokkal szemléltetve.

3. lépés: a korábbi lépéseket valósítsd meg úgy, hogy $4\times4$ helyett egy `n` paraméter megadásával $n\times n$ méretű játékra működjenek.

A feladat megoldása során az A* keresőalgoritmust használtam. 
Az A* lényegében egy továbbgondolt Dijkstra algoritmus, azzal a kiegészítéssel, hogy minden csúcson tárolja a céltól való távolságot. 
A Manhattan-távolság: a puzzle összes elemének a helyétől való távolságának az összege. 
Források:
https://algorithmsinsight.wordpress.com/graph-theory-2/a-star-in-general/implementing-a-star-to-solve-n-puzzle/
https://en.wikipedia.org/wiki/A*_search_algorithm


In [1]:
import numpy as np
from matplotlib import image
from matplotlib import pyplot as plt

In [2]:
def slicing(n):              #Ez a függvény végzi a kép betöltését és feldarabolását
    img = np.array(image.imread('keedyo.jpg'),dtype="uint8")  #Itt töltöm be a képet és alakítom át numpy tömbbé
    img.shape
    i = img.shape[0]
    j = img.shape[1]
    y = i//n
    x = j//n
    parts = []                     #Ebben a változóban tárolom a feldarabolt képeket. 
    s = 1
    for i in range(n):
        parts1 = []
        for j in range(n):
            a = img[i*y:(i+1)*y , j*x:(j+1)*x]     #Itt történik a képek feldarabolása
            parts1.append(a)
        parts.append(parts1)            
    return parts


In [3]:
def goal(n):        #a függvény felelős, hogy a megoldott kirakót elkészíts későbbi referenciák céljából n függvényében
    f = []
    s = 1
    for i in range(n):
        f1 = []
        for j in range(n):
            f1.append(s)
            s += 1
        f.append(f1)
    f[-1][-1] = 0
    return f

In [4]:
def get_position(x,s):             #ez a függvény visszaadja egy elem pozícióját a mátrixban, ha benne van
    for i in range(len(s)):
        for j in range(len(s)):
            if s[i][j]==x:
                position = [i,j]
    return position

In [5]:
#A tili-toli nem mindig kirakható ebben a blokkban vizsgálom a kirakhatóságot.
#Kirakható, ha n páratlan és a cserék (inversions) száma páros
#Kirakható, ha n páros és a cserék száma páratlan és a nulla sorának poziciója alulról számolva páros. 
#Kirakható, ha n páros és a cserék száma páros és a nulla sorának poziciója alulról számolva páratlan.
#Minden más esetben nem lehet megoldani. 

def inversions_count(s):         #Ez a segédfüggvény csak megszámolja a cseréket. 
        inv_count = 0
        puzzle = []
        for i in range(len(s)):
            for j in range(len(s)):
                if s[i][j] != 0:
                    puzzle.append(s[i][j])
            
        for i in range(len(puzzle)):
            for j in range(i + 1, len(puzzle)):
                if puzzle[i] > puzzle[j]:
                    inv_count += 1
        return inv_count
                    
def solvable(s,f):
    if (len(s))%2 == 1 and inversions_count(s)%2 == 0:
        return True
    elif (len(s))%2 == 0 and inversions_count(s)%2 == 1 and (len(s)-get_position(0,s)[0])%2 == 0:
        return True
    elif (len(s))%2 == 0 and inversions_count(s)%2 == 0 and (len(s)-get_position(0,s)[0])%2 == 1:
        return True
    else:
        return False 

In [6]:
def draw_picture(a,img):            #A kirakós aktuális állapota szerint megcsinálja a képet
    s=1
    n = len(a)
    for i in range(n):
        for j in range(n):
            x = a[i][j]
            if x!=0:
                plt.subplot(n,n,s)
                plt.imshow(img[(x-1)//n][(x-1)%n])
                plt.xticks([])
                plt.yticks([])
            s += 1
    plt.show()
    
def step_by_step(solution,img):      #A megoldás kiíratásáért felelős
    for i in solution:
        for j in i:
            print(j)
        draw_picture(i,img)
        print("------------------") 

In [7]:
def swap(a, x1, y1, x2, y2):      #Ez a függvény 2 elem cseréjét végzi a megadott indexek szerint
    x = np.zeros(shape=(len(a),len(a))) #Mindig új mátrixot kell létrehozni különben az adott állapotból egyszerre megcsinálná az összes lehetséges cserét
    for i in range(len(a)):
        for j in range(len(a)):
            if i==x1 and j==y1:
                x[i][j] += a[x2][y2]
            elif i==x2 and j == y2:
                x[i][j] += a[x1][y1]
            else:
                x[i][j] += a[i][j]
    y = x.tolist()                    #Később probléma adódott, amikor numpy tömböt, listák listájával akartam összehasonlítani
    for i in range(len(y)):
        for j in range(len(y)):
            y[i][j] = int(y[i][j])
    return y
    
def steps(a):                   #Ez a függvény a jelenlegi pozicióból elérhető lépéseket gyűjti össze
    steps = []
    index = get_position(0,a)   #nulla poziciója
    i = index[0]  
    j = index[1] 
    if j < len(a) - 1:
        steps.append(swap(a,i,j,i,j+1))  # jobbra lépés
    if i > 0:
        steps.append((swap(a,i,j,i-1,j)))  # felfele lépés 
    if j > 0:
        steps.append(swap(a,i,j,i,j-1))  # balra lépés
    if i < len(a) - 1:
        steps.append(swap(a,i,j,i+1,j))  # lefele lépés
        
    return steps


def distance(a,f):                     # A manhattan távolságot számolja
    d=0
    for i in range(len(a)):
        for j in range(len(a)):
            if a[i][j] != 0:
                index = get_position(a[i][j],f)
                d += abs(index[0]-i) + abs(index[1]-j)
    return d
            
def solveAstar(s,f):                  # Ez a függvény a legfontosabb az A* algoritmus ami kirakja a kirakót. 
    queue = [[distance(s,f), s]]      # Ez a lehetséges útvonalak queueja. Minden útvonal első helyeken írja a távolságot.  
    expanded = []                     # Tárolni akarjuk a látogatott poziciókat, hogy ne legyen ismétlődés
    path = None                       # Ez lesz az út, ami elvezet a megoldáshoz
    
    while queue:
        i = 0
        for j in range(1, len(queue)):
            if queue[i][0] > queue[j][0]:  # minimum manhattan távolságról folytassuk
                i = j

        path = queue[i]                    # min távolságú út lesz a vizsgált út
        queue = queue[:i] + queue[i + 1:]  # itt kivesszük a vizsgált utat a queuból(ne legyen ismétlődés)
        end_node = path[-1]                # utolsó lépése az útnak (jelenleg itt vagyunk)
        
        if end_node == f:                  # ha az utolsó csúcs megegyezik a céllal akkor végeztünk 
            break   
        if end_node in expanded:           # ha az utolsó csúcs benne van az elértekbe, akkor a while ciklus elejére dob minket
            continue
 
        for step in steps(end_node):       # lehetséges további lépések hozzáadása
            if step in expanded:           # ha már vizsgáltuk akkor lépjünk a ciklus elejére
                continue
            new_path = [path[0] + distance(step,f)-distance(end_node,f)] + path[1:] + [step]  # új út létrehozása
            queue.append(new_path)         # új út hozzáadása az eddigiekhez
        expanded.append(end_node)          # az utolsó csúcs hozzáadása az eddig vizsgáltakhoz
    
    solution = path[1:]   # a megoldása az utolsó vizsgált útvonal
    print(len(solution))  # lépések száma
    img = slicing(len(s))
    step_by_step(solution,img)

In [ ]:
start = [[6,2,5,1], [9, 7, 12, 4], [13, 0, 3, 8], [14, 10, 11, 15]]  # itt adjuk meg a kiinduló állapotot
final = goal(len(start))                                             # cél
if (solvable(start,final)):                                          #ha megoldható oldjuk meg
    solveAstar(start,final)  